In [1]:
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.layers import Input, BatchNormalization, SeparableConv2D, Dense, GlobalAveragePooling2D, Activation, AveragePooling2D
from keras.layers import Concatenate, Add
from keras.models import Model

In [3]:
def conv2d_bn(x, f, k=1, s=1, pad='same', use_bias=False):
    x = SeparableConv2D(filters=f, kernel_size=k, strides=s, padding=pad, use_bias=use_bias, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [4]:
def identity_block(ip, f, k=1):
    x = conv2d_bn(ip, f//2, 1, use_bias=True)
    x = conv2d_bn(x, f, k)
    x = Add()([ip,x])
    return x

In [5]:
ip = Input((32,32,3))

In [6]:
#INCEPTION STEM
x = conv2d_bn(ip, 32, 3)
x = conv2d_bn(x, 64, 3)
x = conv2d_bn(x, 128, 3)

In [7]:
#INCEPTION STEM
branch_0 = conv2d_bn(x, 48, 1)
branch_1 = conv2d_bn(x, 24, 1)
branch_1 = conv2d_bn(branch_1, 32, 5)
branch_2 = conv2d_bn(x, 32, 1)
branch_2 = conv2d_bn(branch_2, 48, 3)
branch_2 = conv2d_bn(branch_2, 48, 3)
branch_pool = AveragePooling2D(3, strides=1, padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 32, 1)
branches = [branch_0, branch_1, branch_2, branch_pool]
x = Concatenate()(branches)

In [8]:
#RESNET
x = conv2d_bn(x, 96, 3, 2, use_bias=True)
for i in range(3):
    x = identity_block(x, 96, 3)
x = conv2d_bn(x, 192, 3, 2, use_bias=True)
for i in range(3):
    x = identity_block(x, 192, 3)
x = conv2d_bn(x, 256, 3, 2, use_bias=True)
for i in range(3):
    x = identity_block(x, 256, 3)

In [9]:
x = GlobalAveragePooling2D()(x)
out = Dense(24, activation='softmax', name='Probabilities')(x)
model = Model(ip, out)

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 32, 32, 32)   123         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         separable_conv2d_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
separable_

In [12]:
import os

In [13]:
json_model = model.to_json()
with open('model1.json', 'w') as file:
    file.write(json_model)